In [1]:
import sys

MAIN_DICT = "/gws/nopw/j04/ai4er/users/pn341/earthquake-predictability"
sys.path.append(MAIN_DICT)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data
import tqdm
from sklearn.preprocessing import MinMaxScaler

In [3]:
from utils.dataset import Detrender, SlowEarthquakeDataset, create_sequences
from utils.nn_io import save_model

In [4]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [6]:
columns = {}
dataset = SlowEarthquakeDataset([f"cascadia_{i}_seg" for i in range(0, 14)])
dataset.load()

for i in range(0, 14):
    ds_exp = dataset[f"cascadia_{i}_seg"]
    X, Y, t = ds_exp["X"], ds_exp["Y"], ds_exp["t"]
    columns[f"seg_{i}_avg"] = X.reshape(-1)

ts_data = pd.DataFrame(columns)
ts_data.head()

,seg_0_avg,seg_1_avg,seg_2_avg,seg_3_avg,seg_4_avg,seg_5_avg,seg_6_avg,seg_7_avg,seg_8_avg,seg_9_avg,seg_10_avg,seg_11_avg,seg_12_avg,seg_13_avg
0,1.201987e+06,1.510314e+08,7.801336e+07,6.487482e+06,1.140829e+07,-5.101341e+06,1.611042e+06,-1.209434e+07,1.124978e+07,-1.393699e+07,-7.888723e+06,1.283481e+06,1.418180e+07,3.058038e+06
1,-4.482189e+05,1.256806e+08,9.700589e+07,8.097695e+06,7.052772e+06,-1.201904e+06,1.699715e+06,-2.088164e+07,5.293216e+06,-4.670103e+06,9.272929e+06,7.514658e+06,1.989816e+07,8.899238e+06
2,-1.067187e+06,1.074901e+08,7.106421e+07,1.453868e+07,1.389374e+07,1.186125e+07,9.799695e+06,-1.473196e+07,3.781228e+06,-1.642465e+06,5.886750e+06,2.138463e+06,1.451933e+07,5.668336e+06
3,-4.572402e+05,1.447256e+08,8.363367e+07,1.735795e+07,1.916262e+07,1.163869e+07,1.040321e+07,-1.843907e+07,3.188036e+06,-6.300061e+06,5.185058e+06,1.812858e+06,1.569673e+07,4.202538e+06
4,2.946308e+06,1.787377e+08,8.466346e+07,1.640643e+07,2.450322e+07,1.442252e+07,9.253912e+06,-1.795238e+07,1.242759e+07,-1.243727e+07,-5.960450e+06,-4.436171e+06,1.246743e+07,4.566618e+06


In [7]:
lookback = 200
forecast = 100
windows = 2

X, y = create_sequences(ts_data.values, lookback, forecast)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:", device)

using device: cuda


In [10]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

test_size = windows * forecast
excess = X[:-test_size].shape[0] - forecast * (
    X[:-test_size].shape[0] // forecast
)

X_test, y_test = X[-test_size:], y[-test_size:]
X_train, y_train = X[excess:-test_size], y[excess:-test_size]

# X_train, y_train = detrend_X.fit_transform(X_train), detrend_y.fit_transform(y_train)
# X_test, y_test = detrend_X.transform(X_test), detrend_y.transform(y_test)

X_train_shape, X_test_shape, y_train_shape, y_test_shape = (
    X_train.shape,
    X_test.shape,
    y_train.shape,
    y_test.shape,
)

X_train_sc, X_test_sc = scaler_X.fit_transform(
    X_train.reshape(-1, 1)
), scaler_X.transform(X_test.reshape(-1, 1))
y_train_sc, y_test_sc = scaler_y.fit_transform(
    y_train.reshape(-1, 1)
), scaler_y.transform(y_test.reshape(-1, 1))

y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

X_train_sc = (
    torch.from_numpy(X_train_sc.reshape(*X_train_shape)).float().to(device)
)
y_train_sc = (
    torch.from_numpy(y_train_sc.reshape(*y_train_shape)).float().to(device)
)
X_test_sc = (
    torch.from_numpy(X_test_sc.reshape(*X_test_shape)).float().to(device)
)
y_test_sc = (
    torch.from_numpy(y_test_sc.reshape(*y_test_shape)).float().to(device)
)

loader = data.DataLoader(
    data.TensorDataset(X_train_sc, y_train_sc), shuffle=True, batch_size=64
)